In [23]:
with open('input.txt', 'r', encoding='utf-8') as f:
    data = f.read()

In [24]:
print(data[:100])
chars = sorted(list(set(data)))
print(''.join(chars))
vocab_size = len(chars)
print(f"vocab size: {vocab_size}")

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda s: [itos[t] for t in s]

block = data[:9]
encoded_block = encode(block)
print(encode(block))
decoded_block = decode(encoded_block)
print(decoded_block)


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
[18, 47, 56, 57, 58, 1, 15, 47, 58]
['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't']


In [25]:
import torch
data = torch.tensor(encode(data), dtype=torch.long)
print(f"data shape: {data.shape}\ndata type: {data.dtype}")
print(data[:100])

data shape: torch.Size([1115394])
data type: torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [26]:
# train val split
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]

In [62]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split:str='train', batch_size:int=4):
    data = train_data if split=="train" else val_data
    # Creating 4 random offsets on the dataset
    # data would be batched from those offsets
    # This creates a random sampling of batches
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    # tensor of batch_size, block_size by stacking rows
    x = torch.stack([data[i: i+block_size] for i in ix])
    # Targets tensor offset by 1
    # if input tensor = [42, 37, 65, 80]
    # target tensor = [37, 65, 80, 28]
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

In [40]:
xb, yb = get_batch('train')
print(f"inputs: {xb}")
print(f"targets: {yb}")

inputs: tensor([[53, 44,  1, 57, 54, 43, 43, 41],
        [57, 53, 52, 57,  8,  0,  0, 25],
        [ 1, 42, 43, 44, 43, 41, 58,  1],
        [21,  1, 61, 39, 52, 42, 43, 56]])
targets: tensor([[44,  1, 57, 54, 43, 43, 41, 46],
        [53, 52, 57,  8,  0,  0, 25, 17],
        [42, 43, 44, 43, 41, 58,  1, 53],
        [ 1, 61, 39, 52, 42, 43, 56,  6]])


In [59]:
# Bigram model
import torch.nn as nn
import torch.nn.functional as F


class BiGramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self, x, y=None):
        logits = self.embedding(x)
        # Slight problem with pytorch cross-entropy
        # They expect the input shape to be (B*T, C) and output shape to be (B*T) or flatten the first 2 dims
        if y==None:
            return logits, None
        
        else:
            logits = logits.flatten(0, 1)
            y = y.flatten(0, 1)
            loss = F.cross_entropy(logits, y)

        return logits, loss

    def generate(self, x, max_new_tokens):
        for _ in range(max_new_tokens):
            # Forward pass with X (B, T) gives output -> (B, T+1)
            logits, loss = self(x) # (B, T, C)
            # need to slice the predicted token
            logits = logits[:, -1, :] # (B, C)
            # apply softmax on the channel
            probs = F.softmax(logits, dim=-1)
            # sample from the dist
            # say the probs are 0.06, 0.08, 0.4 ... (65 values)
            # say 0.1 is the higest value and it's corresponding index is 2.
            # so for B=0, x_next = [[2]]
            x_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            x = torch.cat((x, x_next), dim=1) # (B, T+1)

        return x



    
m = BiGramModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
out = m.generate(idx, 100)[0].tolist()
print("".join(decode(out)))


torch.Size([32, 65])
tensor(4.7110, grad_fn=<NllLossBackward0>)

ZL!VEHwJJWNNTyxg
Gca$nhMwZJ3oZ-KEA
lvuFIcbReu;XWr,JBF
T?CV$lxgZEc'cgO!CqNiGH3g;QAkiW.PYwidHMNg'uPS,Q


In [71]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
epochs = 10000

for steps in range(epochs):
    xb, yb = get_batch(batch_size=batch_size)
    optimizer.zero_grad(set_to_none=True)
    _, loss = m.forward(xb, yb)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3846805095672607


In [73]:
idx = torch.zeros((1,1), dtype=torch.long)
out = m.generate(idx, 500)[0].tolist()
print("".join(decode(out)))




O:
All theamino catr?
Whengu Ift thyo ayondoven maly bh y ar a sorouchicrd n hyo y, fetideing lllld allineng, ofiriare ind f l;
Yod ssut'meved
Louge;
Af my ht ESouice t,
Silowist matle
Ane clld t
Toulen mu,
I ftodwies,--be at wste hir;
He outhethmy? tu ay tindorure s wse m meme is G aponege t:


O:
As t'e f irnke t wakes IS:
t m;
KEORCEvelten.


CHiundofr sofivenk,
Bu berourthand ha his s
Red it.
F thy ld oumen be norow ortothouinty pirenomp:
Lo w INGESt noms toorindeo t.
S I oousw fasponjo an
